In [ ]:
import folium
import pandas as pd
import numpy as np
import requests
%matplotlib inline

## Data inspection

**We start out by reading in the data and examining the fields we are about to use**

In [ ]:
df = pd.read_csv("P3_GrantExport.csv", sep=";", index_col=0)
df.sample(5)

**We only keep the necessary columns**

In [ ]:
wanted_columns = ["Project Title", "Funding Instrument", "Institution", "University", "Start Date", "End Date", "Approved Amount"]
df = df[wanted_columns]
df.sample(3)

**Let's check if there are missing values in the Approved Amount column.**

In [ ]:
print("%d approved amounts are NaN" % pd.isnull(df['Approved Amount']).sum())

**However we see that some entries are denoted with a "data not included in P3" string**

In [ ]:
df.loc[121043]["Approved Amount"]

In [ ]:
missing_amount_count = (df["Approved Amount"] == "data not included in P3").sum()
print("{0:.2f}% of approved amounts missing".format(100.0*missing_amount_count / len(df)))

**There's about 1/6th of the data with missing approved amounts. We have no other option than dropping them**

In [ ]:
# Sanity check to see if all missing values are denoted with "data not included in P3"
missing_amounts = pd.to_numeric(df["Approved Amount"], errors="coerce").isnull()
# Check that 
pd.to_numeric(missing_amounts).sum() == missing_amount_count

** After inspecting the rows with missing data, it looks like most of the Institutions are not located in Switzerland. It is also indicated on the documentation page that all mobility fellowship are missing.**

In [ ]:
df[missing_amounts].groupby("Institution").count().sample(5)

**The documentation says that this field is missing for mobility, but we notice that there are other Funding Instruments like "Fellowships for prospective researchers" which lack many Amount values.** 

In [ ]:
df[missing_amounts].groupby("Funding Instrument").count()

In [ ]:
# Dropping project with missing Amounts
print("Length before drop: ", len(df))
df = df[~missing_amounts]
print("Length after drop: ", len(df))

**We see that there are zero-values approved amount. Also, the maximum is 100x bigger than the median**

In [ ]:
df["Approved Amount"] = pd.to_numeric(df["Approved Amount"])
df["Approved Amount"].describe()

In [ ]:
# Biggest project
df[df["Approved Amount"] == df["Approved Amount"].max()]

**Now we look at projects with zero fundings**

In [ ]:
zero_amount_pj = df[df["Approved Amount"] == 0]
zero_amount_pj.sample(5)

**There are only 87 of them**

In [ ]:
len(zero_amount_pj)

**There is a gap in the project numbering, let's see if there is also a gap in the dates (missing period)**

In [ ]:
df["Approved Amount"].plot()

In [ ]:
# Parse the dates into datetime format
df["Start Date"] = pd.to_datetime(df["Start Date"])
df["End Date"] = pd.to_datetime(df["End Date"])

In [ ]:
# Plotting the years
df["Start Date"].apply(lambda d: d.year).plot()

**From our crude analysis, we can see that the data looks inconsistent before 1990, and that the project numbering changed around 2004 but the data looks complete from 1990 to today**

In [ ]:
df["End Date"].apply(lambda d: d.year).plot()

In [ ]:
df["Start Year"] = df["Start Date"].apply(lambda d: d.year)

In [ ]:
df.plot(x='Start Year', y='Approved Amount', style='.')

In [ ]:
# Print the median amount for each year
df.groupby("Start Year").median().plot()

In [ ]:
# Print the total of the amounts for each year
df.groupby("Start Year").sum().plot()

** We see that the yearly budget generally increases over time**

## API calls

**We want to see how much funding went to each Swiss canton. We try to find the canton for each university using the Geonames API.**

In [ ]:
username = "coolestteamada"
GEO_URL = "http://api.geonames.org/searchJSON"

In [ ]:
epfl_geo = requests.get(GEO_URL, params={"username": username, "q": "EPFL"})
epfl_geo.status_code

In [ ]:
epfl_geo = epfl_geo.json()['geonames'][0]
epfl_geo

**We see that the canton is returned in the 'adminCode1' field of the JSON.**

In [ ]:
# seems like we can get the canton easily
epfl_geo['adminCode1']

In [ ]:
def parse_canton(json):
    """Parse the canton code from the geonames json"""
    try:
        return json['geonames'][0]['adminCode1']
    except:
        return None

In [ ]:
def get_uni_canton(uni):
    """Gets the canton code given a University name, using geonames api"""
    r = requests.get(GEO_URL, params={"username": username, "q": uni})
    return parse_canton(r.json())

In [ ]:
# List of all the universities
unis = list(df.groupby("University").groups.keys())
unis[0:5]

In [ ]:
# Number of different universities
len(unis)

**This looks like a reasonable number of calls to the geonames API. We also want to make sure we are not omitting a large amount of projects that might be taking place outside of a university (or with a missing university field).**

In [ ]:
# Percentage of projects with null university field
"{0:.2f}%".format(100 * df["University"].isnull().sum() / len(df))

In [ ]:
# Percentage of funding with null university field
"{0:.2f}%".format(100 * df[df["University"].isnull()]["Approved Amount"].sum() / df["Approved Amount"].sum())

**It is only about 4% of the projects, and 0.4% of the total fundings. So we can omit those projects without a huge loss of information**

In [ ]:
# Try to get cantons for each uni
canton_dict = {}
for uni in unis:
    canton_dict[uni] = get_uni_canton(uni)

In [ ]:
canton_dict.values()

**Uh oh - the geonames API does not work for most of the universities. We will try filling the gaps manually. For some we cannot decide a canton, so we enter None. For multi-canton institutes or universities we assign the canton of their headquarters, if the HQ was given by the organization's website. Otherwise we also assign None.**

In [ ]:
manual_canton_dict = {
    'AO Research Institute - AORI': 'GR',
    'Allergie- und Asthmaforschung - SIAF': 'GR',
    'Berner Fachhochschule - BFH': 'BE',
    'Biotechnologie Institut Thurgau - BITG': 'TG',
    "Centre de rech. sur l'environnement alpin - CREALP": 'VS',
    'EPF Lausanne - EPFL': 'VD',
    'ETH Zürich - ETHZ': 'ZH',
    'Eidg. Anstalt für Wasserversorgung - EAWAG': 'ZH',
    'Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL': 'ZH',
    'Eidg. Hochschulinstitut für Berufsbildung - EHB': 'BE',
    'Eidg. Material und Prüfungsanstalt - EMPA': 'ZH',
    'Ente Ospedaliero Cantonale - EOC': 'TI',
    'Fachhochschule Kalaidos - FHKD': 'ZH',
    'Fachhochschule Nordwestschweiz (ohne PH) - FHNW': 'SO',
    'Fachhochschule Ostschweiz - FHO': 'SG',
    'Facoltà di Teologia di Lugano - FTL': 'TI',
    'Fernfachhochschule Schweiz (Mitglied SUPSI) - FFHS': 'VS',
    'Firmen/Privatwirtschaft - FP': None,
    'Forschungsanstalten Agroscope - AGS': None, # NOTE spread across many cantons
    'Forschungsinstitut für Opthalmologie - IRO': 'VS',
    'Forschungsinstitut für biologischen Landbau - FIBL': 'AG',
    'Forschungskommission SAGW': 'BE',
    'Franklin University Switzerland - FUS': 'TI',
    'Friedrich Miescher Institute - FMI': 'BS',
    'HES de Suisse occidentale - HES-SO': None, # NOTE in Delemont but spread among 7 cantons
    'Haute école pédagogique BE, JU, NE - HEPBEJUNE': None, # NOTE no assignment; there are 3 campuses, one on each canton.
    'Haute école pédagogique du canton de Vaud - HEPL': 'VD',
    'Haute école pédagogique fribourgeoise - HEPFR': 'FR',
    'Hochschule Luzern - HSLU': 'LU',
    'Idiap Research Institute - IDIAP': 'VS',
    'Inst. Suisse de Spéléologie et Karstologie - ISSKA': 'NE',
    'Inst. de Hautes Etudes Internat. et du Dév - IHEID': 'GE',
    'Inst. universit. romand de Santé au Travail - IST': 'VD',
    'Institut Universitaire Kurt Bösch - IUKB': 'VS',
    'Institut für Kulturforschung Graubünden - IKG': 'GR',
    'Instituto Ricerche Solari Locarno - IRSOL': 'TI',
    'Interkant. Hochschule für Heilpädagogik ZH - HfH': 'ZH',
    'Istituto Svizzero di Roma - ISR': None, # NOTE in Rome, Italy!
    'Kantonsspital St. Gallen - KSPSG': 'SG',
    'NPO (Biblioth., Museen, Verwalt.) - NPO': None,
    'Nicht zuteilbar - NA': None,
    'Paul Scherrer Institut - PSI': 'AG',
    'Physikal.-Meteorolog. Observatorium Davos - PMOD': 'GR',
    'Pädag. Hochschule Tessin (Teilschule SUPSI) - ASP': 'TI',
    'Pädagogische Hochschule Bern - PHBern': 'BE',
    'Pädagogische Hochschule Graubünden - PHGR': 'GR',
    'Pädagogische Hochschule Luzern - PHLU': 'LU',
    'Pädagogische Hochschule Nordwestschweiz - PHFHNW': 'AG',
    'Pädagogische Hochschule Schaffhausen - PHSH': 'SH',
    'Pädagogische Hochschule Schwyz - PHSZ': 'SZ',
    'Pädagogische Hochschule St. Gallen - PHSG': 'SG',
    'Pädagogische Hochschule Thurgau - PHTG': 'TG',
    'Pädagogische Hochschule Wallis - PHVS': 'VS',
    'Pädagogische Hochschule Zug - PHZG': 'ZG',
    'Pädagogische Hochschule Zürich - PHZFH': 'ZH',
    'Robert Walser-Stiftung Bern - RWS': 'BE',
    'SUP della Svizzera italiana - SUPSI': 'TI',
    'Schweiz. Hochschule für Logopädie Rorschach - SHLR': 'SG',
    'Schweiz. Institut für Kunstwissenschaft - SIK-ISEA': 'ZH',
    'Schweizer Kompetenzzentrum Sozialwissensch. - FORS': 'VD',
    'Schweizer Paraplegiker Forschung - SPF': 'LU',
    'Staatsunabh. Theologische Hochschule Basel - STHB': 'BS',
    'Swiss Center for Electronics and Microtech. - CSEM': 'NE',
    'Swiss Institute of Bioinformatics - SIB': 'VD',
    'Università della Svizzera italiana - USI': 'TI',
    'Universität Basel - BS': 'BS', #api
    'Universität Bern - BE': 'BE', #api
    'Universität Luzern - LU': 'LU',
    'Universität St. Gallen - SG': 'SG',
    'Universität Zürich - ZH': 'ZH', #api
    'Université de Fribourg - FR': 'FR', #api
    'Université de Genève - GE': 'GE',
    'Université de Lausanne - LA': 'VD',
    'Université de Neuchâtel - NE': 'NE', #api
    'Weitere Institute - FINST': None,
    'Weitere Spitäler - ASPIT': None,
    'Zürcher Fachhochschule (ohne PH) - ZFH': 'ZH'
}

In [ ]:
# We create a DataFrame from the dict
canton_df = pd.DataFrame.from_dict(manual_canton_dict, orient='index')
canton_df.columns = ['Canton']
canton_df.sample(5)

In [ ]:
# And now we merge the two DataFrame to add the canton code for each project
df_with_canton = pd.merge(left=df, right=canton_df, left_on='University', right_index=True)
df_with_canton.sample(5)


In [ ]:
# Total amount with no canton assigned
total_amt_no_canton = df_with_canton[df_with_canton['Canton'].isnull()]['Approved Amount'].sum()

# Total amount
total_amt = df_with_canton['Approved Amount'].sum()
print("{0:.2f}% of funds without a canton".format(100.0* total_amt_no_canton/total_amt))

**We see that only 5% of the funding in our table went to universities with no canton assigned. So we now drop all projects without a canton**

In [ ]:
print("Len before drop: %d" % len(df_with_canton))
df_projects_with_canton = df_with_canton.dropna(subset=['Canton'])
print("Len after drop: %d" % len(df_projects_with_canton))

**We dropped roughly 5000 projects that did not have a canton. Time to make our funding map!**

In [ ]:
# Now we sum the total amount assigned to each canton
funding_by_canton = df_projects_with_canton.groupby('Canton').sum()
funding_by_canton = funding_by_canton[['Approved Amount']]
funding_by_canton

In [ ]:
funding_by_canton.describe()

**We see however that we have 18 cantons instead of 26. Let's fill the missing cantons and give them zero approved amounts**

In [ ]:
swiss_cantons_list = ["AG", "AI", "AR", "BE", "BL", "BS", "FR", "GE", "GL", "GR", "JU", "LU", "NE", "NW", "OW", "SG", "SH", "SO", "SZ", "TG", "TI", "UR", "VD", "VS", "ZG", "ZH"]
df_all_canton = pd.DataFrame(swiss_cantons_list, columns=["Canton"])
df_all_canton['Approved Amount'] = 0
df_all_canton = df_all_canton.set_index('Canton')
df_all_canton.head()

**The dataframe below contains the funding total for every canton.**

In [ ]:
# Add missing cantons
df_to_map = funding_by_canton.add(df_all_canton, fill_value=0)
# Also reset the index in order for folium to work (it needs a proper column)
df_to_map = df_to_map.reset_index()

In [ ]:
df_to_map.head()

** Let's test folium's choropleth with the funding total for each canton**

In [ ]:
# We remove the warnings because choropleth always yields an unuseful FutureWarning
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

In [ ]:
geo_path = 'ch-cantons.topojson.json'
    

switzerland = folium.Map(location=[46.57, 8], zoom_start=8)
switzerland.choropleth(geo_path=geo_path, 
                     data=df_to_map,
                     columns=['Canton', 'Approved Amount'],
                     key_on='feature.id',
                     topojson='objects.cantons',
                     fill_color='YlGn'
                    )
switzerland

** A few cantons (Zürich, Vaud, Geneva, and Bern) dominate the funding landscape and the rest barely show up on the Choropleth. 
We plot using a log scale for another perspective on the funding differences**

In [ ]:
# Apply a log to the amounts. We add 1 to avoid negative values from the log
df_to_map['Log Approved Amount'] = np.log(df_to_map['Approved Amount'] + 1)
df_to_map.head()

In [ ]:
switzerland_log = folium.Map(location=[46.57, 8], zoom_start=8)
switzerland_log.choropleth(geo_path=geo_path, 
                     data=df_to_map,
                     columns=['Canton', 'Log Approved Amount'],
                     key_on='feature.id',
                     topojson='objects.cantons',
                     fill_color='YlGn'
                    )
switzerland_log

**Save these maps as HTML (for our reviewers)**

In [ ]:
switzerland.save("funding_map_switzerland.html")
switzerland_log.save("funding_map_switzerland_logscale.html")